In [0]:
##import all:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [0]:
# load dataset
url_input = "https://raw.githubusercontent.com/AigerimJ/heart_disease/master/heart/train/train_values.csv?token=ATmRgOC2Z57N9x2_xJ9Uj_9rWjrGvjPrks5cleXwwA%3D%3D"
# df = pandas.read_csv(url, delim_whitespace=True, header=None)
url_output = "https://raw.githubusercontent.com/AigerimJ/heart_disease/master/heart/train/train_labels.csv?token=ATmRgKqn1MgBzsqPavDHl89xvsH9-dPUks5clfqiwA%3D%3D"


df_input = pandas.read_csv(url_input)
df_output = pandas.read_csv(url_output)

dataset_input = df_input.values
dataset_output = df_output.values

# split into input (X) and output (Y) variables
# X = dataset_input[:,1:14]
# Y = dataset_output[:,1]

for i in range(len(X)):
  if (X[i][1] == 'normal'):
    X[i][1] = 0
    
  if (X[i][1] == 'fixed_defect'):
    X[i][1] = 1
    
  if (X[i][1] == 'reversible_defect'):
    X[i][1] = 2

print(" X = ",X)
print("\nY = ",Y)

 X =  [[1 0 128 ... 45 170 0]
 [2 0 110 ... 54 158 0]
 [1 0 125 ... 77 162 1]
 ...
 [1 0 152 ... 67 172 0]
 [1 2 132 ... 57 168 1]
 [1 2 120 ... 46 144 0]]

Y =  [0 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0
 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0
 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0
 0 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 1 1 0 0 1
 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1]


# Training a Baseline model

In [0]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(3, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [0]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)

print(results)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[-0.25469238 -0.42911067 -0.27984881 -0.30484116 -0.32337605 -0.24752221
 -0.31512121 -0.31280398 -0.23373444 -0.24476523]
Results: -0.29 (0.05) MSE


# Evaluating model with standardized dataset

In [0]:
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by 

Standardized: -0.15 (0.05) MSE


# Using the trained models to predict result on new data

In [0]:
pipeline.fit(X,Y) # Fit model

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fe3cf24ac50>)])

In [0]:
## data to test:

url_test = "https://raw.githubusercontent.com/AigerimJ/heart_disease/master/heart/train/train_values_labels.csv?token=ATmRgJBfnKiWhBF12_5Yu0mR4AQeg5NFks5clgYrwA%3D%3D"
df = pandas.read_csv(url_test)
dataset = df.values

# split into input (X) and output (Y) variables
X_new = dataset[:,1:14]
Y_new = dataset[:,14]

for i in range(len(X_new)):
  if (X_new[i][1] == 'normal'):
    X_new[i][1] = 0
    
  if (X_new[i][1] == 'fixed_defect'):
    X_new[i][1] = 1
    
  if (X_new[i][1] == 'reversible_defect'):
    X_new[i][1] = 2

# print(" X_new = ",X_new)

# print(X_new[0].shape)
# print(X_new[2].shape)



In [0]:
for i in range(len(X_new)):
  pipeline.predict([X_new[i]])
 

In [0]:
Y_pred = numpy.empty((len(X_new), 1))

for i in range(len(X_new)):
  Y_pred[i] += pipeline.predict([X_new[i]])
  
#   if (Y_pred[i] < 0.5):
#     Y_pred[i] = 0
#   else: 
#     Y_pred[i] = 1

  print(i,") Labeled case: %2f" %Y_new[i], " Prediction: %2f" %Y_pred[i])


# Y_pred = pipeline.predict([X_new[0]])
# print("Labeled case: %2f" %Y_new[0])
# print("Prediction: %2f" %Y_pred)

0 ) Labeled case: 0.000000  Prediction: 0.594996
1 ) Labeled case: 0.000000  Prediction: 0.235547
2 ) Labeled case: 1.000000  Prediction: 0.505713
3 ) Labeled case: 1.000000  Prediction: 0.279031
4 ) Labeled case: 1.000000  Prediction: 0.743861
5 ) Labeled case: 1.000000  Prediction: 0.748939
6 ) Labeled case: 0.000000  Prediction: 0.207870
7 ) Labeled case: 1.000000  Prediction: 0.609178
8 ) Labeled case: 1.000000  Prediction: 0.489798
9 ) Labeled case: 1.000000  Prediction: 0.591172
10 ) Labeled case: 0.000000  Prediction: 0.053025
11 ) Labeled case: 0.000000  Prediction: -0.113154


# A larger network

In [0]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu')) # input layer; size 13
    model.add(Dense(6, kernel_initializer='normal', activation='relu')) # hidden layer; size 6
    model.add(Dense(1, kernel_initializer='normal')) # output layer; size 1 with no activation since it's a regression problem
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [0]:
# evaluation on the larger net
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by 

Larger: -0.17 (0.06) MSE


In [0]:
pipeline.fit(X,Y) # Fit model

for i in range(len(X_new)):
  pipeline.predict([X_new[i]])
  
Y_pred = numpy.empty((len(X_new), 1))

for i in range(len(X_new)):
  Y_pred[i] += pipeline.predict([X_new[i]])
  
#   if (Y_pred[i] < 0.5):
#     Y_pred[i] = 0
#   else: 
#     Y_pred[i] = 1

  print(i,") Labeled case: %2f" %Y_new[i], " Prediction: %2f" %Y_pred[i])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0 ) Labeled case: 0.000000  Prediction: 1.235350
1 ) Labeled case: 0.000000  Prediction: 1.227033
2 ) Labeled case: 1.000000  Prediction: 2.793457
3 ) Labeled case: 1.000000  Prediction: 2.298227
4 ) Labeled case: 1.000000  Prediction: 3.139744
5 ) Labeled case: 1.000000  Prediction: 2.981715
6 ) Labeled case: 0.000000  Prediction: 0.372286
7 ) Labeled case: 1.000000  Prediction: 2.967112
8 ) Labeled case: 1.000000  Prediction: 3.075993
9 ) Labeled case: 1.000000  Prediction: 2.600524
10 ) Labeled case: 0.000000  Prediction: 0.390203
11 ) Labeled case: 0.000000  Prediction: -0.197653


##Wider network


In [0]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
  
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by 

Wider: -0.21 (0.06) MSE


In [0]:
pipeline.fit(X,Y) # Fit model

for i in range(len(X_new)):
  pipeline.predict([X_new[i]])
  
Y_pred = numpy.empty((len(X_new), 1))

for i in range(len(X_new)):
  Y_pred[i] += pipeline.predict([X_new[i]])
  
#   if (Y_pred[i] < 0.5):
#     Y_pred[i] = 0
#   else: 
#     Y_pred[i] = 1

  print(i,") Labeled case: %2f" %Y_new[i], " Prediction: %2f" %Y_pred[i])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0 ) Labeled case: 0.000000  Prediction: 0.519476
1 ) Labeled case: 0.000000  Prediction: 0.396671
2 ) Labeled case: 1.000000  Prediction: 1.499646
3 ) Labeled case: 1.000000  Prediction: 1.206733
4 ) Labeled case: 1.000000  Prediction: 1.774449
5 ) Labeled case: 1.000000  Prediction: 1.890537
6 ) Labeled case: 0.000000  Prediction: 0.354639
7 ) Labeled case: 1.000000  Prediction: 1.577021
8 ) Labeled case: 1.000000  Prediction: 1.256994
9 ) Labeled case: 1.000000  Prediction: 1.438690
10 ) Labeled case: 0.000000  Prediction: 0.124271
11 ) Labeled case: 0.000000  Prediction: -0.145587
